## Predicting BreastScreen NSW Attendance to Improve Participation

This project aims to determine the accuracy with which attendance on next screening episode can be predicted, classifying women as either "regular" or "lapsed".

In [4]:
# Import required packages
import pandas as pd
import pandas_profiling
import numpy as np
import xgboost as xgb
import graphviz
import matplotlib.pyplot as plt
from sklearn import model_selection, metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from joblib import dump, load
import time

In [5]:
# Import the data
dtypes = {
'CountryOfBirth':                           'category',
'MainLanguage':                             'category',
'IndigenousStatus':                         'category',
'RemotenessArea':                           'category',
'RelativeSocioEconomicDisadvantageDecile':  'category',
'HistoryPreviousCancer':                    bool,
'HistoryFamilyCancer':                      bool,
'TotalEpisodes':                            int,
'HasBeenDNA':                               bool,
'HadAssessment':                            bool,
'HadNeedleBiopsy':                          bool,
'HadTechRecall':                            bool,
'AgeAtMostRecentEpisode':                   int,
'DistanceKms':                              float,
'MonthMostRecentScreening':                 'category',
'DayOfWeekMostRecentScreening':             'category',
'HourOfDayMostRecentScreening':             'category',
'VenueTypeMostRecentScreening':             'category',
'FilmsTakenMostRecentScreening':            int,
'DaysFromAttendanceToResultSent':           int,
'LapsedRegular3rdMostRecentEpisode':        'category',
'LapsedRegular2ndMostRecentEpisode':        'category',
'LapsedRegularMostRecentEpisode':           'category'}

df = pd.read_csv('Data extraction - 1000 observations scrambled.csv', dtype = dtypes)

In [6]:
# How many rows and columns?
df.shape

(824812, 23)

In [7]:
# Inspect the first few rows
df.head()

,CountryOfBirth,MainLanguage,IndigenousStatus,RemotenessArea,RelativeSocioEconomicDisadvantageDecile,HistoryPreviousCancer,HistoryFamilyCancer,TotalEpisodes,HasBeenDNA,HadAssessment,...,DistanceKms,MonthMostRecentScreening,DayOfWeekMostRecentScreening,HourOfDayMostRecentScreening,VenueTypeMostRecentScreening,FilmsTakenMostRecentScreening,DaysFromAttendanceToResultSent,LapsedRegular3rdMostRecentEpisode,LapsedRegular2ndMostRecentEpisode,LapsedRegularMostRecentEpisode
0,INDIA,Other (please specify),Non-indigenous,Major Cities of Australia,3,False,False,2,False,False,...,0.0,12,5,9,Fixed,4,4,NaN,NaN,Lapsed
1,LEBANON,Arabic,Non-indigenous,Major Cities of Australia,3,False,False,5,True,False,...,0.0,8,4,9,Fixed,4,6,NaN,Lapsed,Regular
2,AUSTRALIA,English Only,Non-indigenous,Major Cities of Australia,10,False,True,3,False,False,...,0.0,11,3,17,Fixed,4,14,NaN,NaN,Regular
3,AUSTRALIA,English Only,Non-indigenous,Major Cities of Australia,10,False,False,2,False,False,...,4.0,3,3,9,Fixed,4,6,NaN,NaN,Lapsed
4,AUSTRALIA,English Only,Non-indigenous,Major Cities of Australia,10,False,False,8,False,False,...,2.0,12,4,9,Fixed,4,5,Regular,Regular,Regular


In [8]:
# Verify data types
df.dtypes

CountryOfBirth                             category
MainLanguage                               category
IndigenousStatus                           category
RemotenessArea                             category
RelativeSocioEconomicDisadvantageDecile    category
HistoryPreviousCancer                          bool
HistoryFamilyCancer                            bool
TotalEpisodes                                 int64
HasBeenDNA                                     bool
HadAssessment                                  bool
HadNeedleBiopsy                                bool
HadTechRecall                                  bool
AgeAtMostRecentEpisode                        int64
DistanceKms                                 float64
MonthMostRecentScreening                   category
DayOfWeekMostRecentScreening               category
HourOfDayMostRecentScreening               category
VenueTypeMostRecentScreening               category
FilmsTakenMostRecentScreening                 int64
DaysFromAtte

In [9]:
# Generate pandas_profiling output for EDA
pfr = pandas_profiling.ProfileReport(df)
pfr.to_file("pandas-profiling output.html")

/anaconda3/envs/ml/lib/python3.7/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [10]:
# No need to clean missing data, normalise numeric features or remove correlated features,
# as boosted trees (which is what we will be using) are very robust to these potential data problems

# Perform one-hot encoding of all categorical columns

# CountryOfBirth                             
one_hot = pd.get_dummies(df['CountryOfBirth'], prefix = 'CountryOfBirth')
df = df.drop('CountryOfBirth',axis = 1)
df = df.join(one_hot)

# MainLanguage                             
one_hot = pd.get_dummies(df['MainLanguage'], prefix = 'MainLanguage')
df = df.drop('MainLanguage',axis = 1)
df = df.join(one_hot)

# IndigenousStatus                             
one_hot = pd.get_dummies(df['IndigenousStatus'], prefix = 'IndigenousStatus')
df = df.drop('IndigenousStatus',axis = 1)
df = df.join(one_hot)

# RemotenessArea                             
one_hot = pd.get_dummies(df['RemotenessArea'], prefix = 'RemotenessArea')
df = df.drop('RemotenessArea',axis = 1)
df = df.join(one_hot)

# RelativeSocioEconomicDisadvantageDecile                             
one_hot = pd.get_dummies(df['RelativeSocioEconomicDisadvantageDecile'], prefix = 'RelativeSocioEconomicDisadvantageDecile')
df = df.drop('RelativeSocioEconomicDisadvantageDecile',axis = 1)
df = df.join(one_hot)

# MonthMostRecentScreening                             
one_hot = pd.get_dummies(df['MonthMostRecentScreening'], prefix = 'MonthMostRecentScreening')
df = df.drop('MonthMostRecentScreening',axis = 1)
df = df.join(one_hot)

# DayOfWeekMostRecentScreening                             
one_hot = pd.get_dummies(df['DayOfWeekMostRecentScreening'], prefix = 'DayOfWeekMostRecentScreening')
df = df.drop('DayOfWeekMostRecentScreening',axis = 1)
df = df.join(one_hot)

# HourOfDayMostRecentScreening                             
one_hot = pd.get_dummies(df['HourOfDayMostRecentScreening'], prefix = 'HourOfDayMostRecentScreening')
df = df.drop('HourOfDayMostRecentScreening',axis = 1)
df = df.join(one_hot)

# VenueTypeMostRecentScreening                             
one_hot = pd.get_dummies(df['VenueTypeMostRecentScreening'], prefix = 'VenueTypeMostRecentScreening')
df = df.drop('VenueTypeMostRecentScreening',axis = 1)
df = df.join(one_hot)

# LapsedRegular3rdMostRecentEpisode                             
one_hot = pd.get_dummies(df['LapsedRegular3rdMostRecentEpisode'], prefix = 'LapsedRegular3rdMostRecentEpisode')
df = df.drop('LapsedRegular3rdMostRecentEpisode',axis = 1)
df = df.join(one_hot)

# LapsedRegular2ndMostRecentEpisode                             
one_hot = pd.get_dummies(df['LapsedRegular2ndMostRecentEpisode'], prefix = 'LapsedRegular2ndMostRecentEpisode')
df = df.drop('LapsedRegular2ndMostRecentEpisode',axis = 1)
df = df.join(one_hot)

df.columns.tolist()

['HistoryPreviousCancer',
 'HistoryFamilyCancer',
 'TotalEpisodes',
 'HasBeenDNA',
 'HadAssessment',
 'HadNeedleBiopsy',
 'HadTechRecall',
 'AgeAtMostRecentEpisode',
 'DistanceKms',
 'FilmsTakenMostRecentScreening',
 'DaysFromAttendanceToResultSent',
 'LapsedRegularMostRecentEpisode',
 'CountryOfBirth_AFGHANISTAN',
 'CountryOfBirth_ALBANIA',
 'CountryOfBirth_ALGERIA',
 'CountryOfBirth_ARGENTINA',
 'CountryOfBirth_ARMENIA',
 'CountryOfBirth_ARUBA',
 'CountryOfBirth_AUSTRALIA',
 'CountryOfBirth_AUSTRIA',
 'CountryOfBirth_BANGLADESH',
 'CountryOfBirth_BELGIUM',
 'CountryOfBirth_BHUTAN',
 'CountryOfBirth_BOLIVIA, PLURINATIONAL STATE OF',
 'CountryOfBirth_BOSNIA AND HERZEGOVINA',
 'CountryOfBirth_BRAZIL',
 'CountryOfBirth_BRUNEI DARUSSALAM',
 'CountryOfBirth_BULGARIA',
 'CountryOfBirth_CAMBODIA',
 'CountryOfBirth_CANADA',
 'CountryOfBirth_CENTRAL AFRICAN REPUBLIC',
 'CountryOfBirth_CHILE',
 'CountryOfBirth_CHINA',
 'CountryOfBirth_COCOS (KEELING) ISLANDS',
 'CountryOfBirth_COLOMBIA',
 'Coun

In [11]:
df.shape

(824812, 320)

In [12]:
# Split the data into X and y
TargetVariable = 'LapsedRegularMostRecentEpisode'
X = df.loc[:, df.columns != TargetVariable]
y = np.ravel(df.loc[:, df.columns == TargetVariable])
print(X.shape)
print(y.shape)

(824812, 319)
(824812,)


In [13]:
# Encode the string target class values ("regular" or "lapsed") as integers in a numpy array
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)

In [14]:
# Split the data set into 70% training, 30% test, using the encoded numpy array
seed = 7
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, label_encoded_y, test_size=test_size, random_state=seed)

In [15]:
# Attempt 1: call XGBClassifier with no parameters, using all of the default values, to measure the baseline accuracy

start_time = time.time()

# Fit the model to training data
model_baseline = xgb.XGBClassifier()
model_baseline.fit(X_train, y_train)
print(model_baseline)
# Output:
#     XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#                   colsample_bynode=1, colsample_bytree=1, gamma=0,
#                   learning_rate=0.1, max_delta_step=0, max_depth=3,
#                   min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
#                   nthread=None, objective='binary:logistic', random_state=0,
#                   reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#                   silent=None, subsample=1, verbosity=1)

# Make predictions for the test data
y_pred = model_baseline.predict(X_test)
predictions = [round(value) for value in y_pred]

# Evaluate the predictions made using the test data
accuracy = accuracy_score(y_test, predictions)
print("Untuned accuracy: %.2f%%" % (accuracy * 100.0))
# Output:
#     Untuned accuracy: 78.71%
#     --- 787.0630960464478 seconds ---
# So, our baseline accuracy on the test dataset is 78.71% - can we do better?

# Save the model to disk using joblib’s replacement of pickle (dump & load), which is more efficient on objects
# that carry large numpy arrays internally as is often the case for fitted scikit-learn estimators
dump(model_baseline, 'model_baseline.joblib')

print("--- %s seconds ---" % (time.time() - start_time))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
Untuned accuracy: 78.71%
--- 787.0630960464478 seconds ---


In [16]:
# Attempt 2: See if we can improve this using XGBoost's built-in cross-validation capabilities

start_time = time.time()

# Create the DMatrix required for the xgboost cv method, using some minimal parameters
dmatrix = xgb.DMatrix(data=X_train, label=y_train)
params={"objective":"binary:logistic","max_depth":4}
model_cv = xgb.cv(dtrain=dmatrix, params=params, nfold=4, num_boost_round=10, metrics=["error","auc"],
                    as_pandas=True)

print("Untuned cross-validated accuracy: %.2f%%" %(((1 - model_cv["test-error-mean"]).iloc[-1]) * 100.0))
print("Untuned cross-validated AUC: %.4f" %(((model_cv["test-auc-mean"]).iloc[-1])))
# Output:
#     Untuned cross-validated accuracy: 78.68%
# which is slightly worse than the baseline accuracy of 78.71%
#     Untuned cross-validated AUC: 0.7880
#     --- 360.53999495506287 seconds ---

print("--- %s seconds ---" % (time.time() - start_time))

# Save the model to disk
dump(model_cv, 'model_cv.joblib')

Untuned cross-validated accuracy: 78.68%
Untuned cross-validated AUC: 0.7880
--- 360.53999495506287 seconds ---


['model_cv.joblib']

In [17]:
# Attempt 3: See if we can improve this using a higher number of boosting rounds
# with automated boosting round selection using early_stopping

start_time = time.time()

# Create the DMatrix required for the xgboost cv method, using some minimal parameters
dmatrix = xgb.DMatrix(data=X_train, label=y_train)
params={"objective":"binary:logistic","max_depth":4}
model_cv_early_stopping = xgb.cv(dtrain=dmatrix, params=params, early_stopping_rounds=10, nfold=4,
                                   num_boost_round=50, metrics=["error","auc"], seed=0, as_pandas=True)

print("Cross-validated early_stopping accuracy: %.2f%%" %(((1 - model_cv_early_stopping["test-error-mean"]).iloc[-1]) * 100.0))
print("Cross-validated early_stopping AUC: %.4f" %(((model_cv_early_stopping["test-auc-mean"]).iloc[-1])))
# Output:
#     Cross-validated early_stopping accuracy: 79.42%
# which is slightly higher than the baseline accuracy of 78.71%
#     Cross-validated early_stopping AUC: 0.8010
#--- 1507.6005401611328 seconds ---
# which is slightly higher than the untuned cross-validated AUC of 0.7880

print("--- %s seconds ---" % (time.time() - start_time))

# Save the model to disk
dump(model_cv_early_stopping, 'model_cv_early_stopping.joblib')

Cross-validated early_stopping accuracy: 79.42%
Cross-validated early_stopping AUC: 0.8010
--- 1507.6005401611328 seconds ---


['model_cv_early_stopping.joblib']

In [18]:
# Some frequently tuned XGBoost parameters for the tree base learner:
#
# learning rate (eta):
#     Affects how quickly the model fits the residual error using additional base learners.
#     A low learning rate will require more boosting rounds to achieve the same reduction in residual error as an
#     XGBoost model with a high learning rate.
# gamma:
#     Minimum loss reduction to create new tree split, which affects how strongly regularised the trained model will
#     be.
# lambda:
#     L2 regularisation on leaf weights, which affects how strongly regularised the trained model will be.
# alpha:
#     L1 regularisation on leaf weights, which affects how strongly regularised the trained model will be.
# max_depth:
#     Maximum depth per tree.
#     Must be a positive integer value.
#     Affects how deeply each tree is allowed to grow during any given boosting round.
# subsample:
#     Percentage of samples used per tree.
#     Must be a value between 0 and 1.
#     The fraction of the total training set that can be used for any given boosting round.
#     If the value is low, then the fraction of training data used per boosting round would be low and may run into
#     underfitting problems, while a value that is very high can lead to overfitting.
# colsample_bytree:
#     Percentage of features used per tree.
#     The fraction of features that can be selected from during any given boosting round.
#     Must be a value between 0 and 1.
#     A large value means that almost all features can be used to build a tree during a given boosting round,
#     while a small value means that the fraction of features that can be selected from is very small.
#     Smaller values can be thought of as providing additional regularisation to the model,
#     while using all columns may overfit a trained model.
# num_boost_round:
#     Number of boosting rounds.
#     The number of trees to be built or the number of base learners to be constructed.

In [21]:
# Attempt 4: See if we can improve this by tuning a few of the above hyperparameters using GridSearch

start_time = time.time()

# Create the parameter grid
gbm_param_grid = {
    "colsample_bytree":[0.3, 0.7],
    "n_estimators": [50],
    "max_depth": [2, 5]
}

# Instantiate the classifier
gbm = xgb.XGBClassifier()

# Perform grid search
model_grid_search = GridSearchCV(param_grid=gbm_param_grid, estimator=gbm, scoring="roc_auc", cv=4,
                              n_jobs=-1, verbose=1)

# Fit model_grid_search to the data
model_grid_search.fit(X_train, y_train)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", model_grid_search.best_params_)
print("Best AUC found: %.4f", %(model_grid_search.best_score_))

# Output:
#     Fitting 4 folds for each of 4 candidates, totalling 16 fits
#     [Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
#     [Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 25.3min finished
#     Best parameters found:  {'colsample_bytree': 0.7, 'max_depth': 5, 'n_estimators': 50}
#     Best AUC found:  0.7997
#     --- 1969.9332740306854 seconds ---
# This AUC is slightly worse than the cross-validated early_stopping AUC of 0.8010

print("--- %s seconds ---" % (time.time() - start_time))

# Save the model to disk
dump(model_grid_search, 'model_grid_search.joblib')

Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 25.3min finished


Best parameters found:  {'colsample_bytree': 0.7, 'max_depth': 5, 'n_estimators': 50}
Best AUC found: %.4f 0.7997267093920994
--- 1969.9332740306854 seconds ---


['model_grid_search.joblib']

In [74]:
# Define a function to create XGBoost models and perform cross-validation
def modelfit(alg, data, label, useTrainCV = True, cv_folds = 5, early_stopping_rounds = 50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(data=data, label=label)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    # Fit the algorithm on the data
    alg.fit(data, label, eval_metric='auc', verbose=True)
        
    # Predict training set
    dtrain_predictions = alg.predict(data)
    dtrain_predprob = alg.predict_proba(data)[:,1]
        
    # Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(label, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(label, dtrain_predprob))
    print("Best Iteration: {}".format(alg.get_booster().best_iteration))  
    print("Best ntree limit: {}".format(alg.get_booster().best_ntree_limit))
    
    feat_imp = pd.Series(alg.get_booster().get_fscore()).nlargest(50).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Top 50 Feature Importances')
    plt.ylabel('Feature Importance Score')
    plt.tight_layout()

In [79]:
# Attempt 5: Try a step-by-step approach to hyperparameter tuning

start_time = time.time()

# Step 1: Fix the learning rate and the number of estimators
model_step_by_step = xgb.XGBClassifier(
    learning_rate = 0.1,
    n_estimators = 1000,
    max_depth = 5,
    min_child_weight = 1,
    gamma = 0,
    subsample = 0.8,
    colsample_bytree = 0.8,
    objective = 'binary:logistic',
    nthread = 4,
    scale_pos_weight = 1,
    seed = 27)

modelfit(model_step_by_step, X_train, y_train)

# Save the Feature Importances bar chart to a file
plt.savefig('Feature Importances - initial.png')

# Output:
#     Model Report
#     Accuracy : 0.8069
#     AUC Score (Train): 0.820695
#     Best Iteration: 787
#     Best ntree limit: 788
#     --- 12947.70002579689 seconds ---
# So, we've improved the AUC from 0.8010 to 0.820695. Let's keep tuning to see if we can improve this.

print("--- %s seconds ---" % (time.time() - start_time))

# Save the model to disk
dump(model_step_by_step, 'model_step_by_step.joblib')


Model Report
Accuracy : 0.8069
AUC Score (Train): 0.820695
--- 12947.70002579689 seconds ---


['model_step_by_step.joblib']

In [ ]:
# Step 2: Tune max_depth and min_child_weight, as they will have the highest impact on model outcome.
# Use the optimal n_estimators value calculated in the previous step (788).
# To start with, set wider ranges for max_depth and min_child_weight and then perform another iteration for
# smaller ranges.

start_time = time.time()

param_step_by_step_test1 = {
    'max_depth': range(3, 10, 2),
    'min_child_weight': range(1, 6, 2)
}
gsearch_step_by_step_test1 = GridSearchCV(estimator = xgb.XGBClassifier(learning_rate=0.1, n_estimators = 788,
                                          max_depth = 5, min_child_weight = 1, gamma = 0, subsample = 0.8,
                                          colsample_bytree = 0.8, objective = 'binary:logistic', nthread = 4,
                                          scale_pos_weight = 1, seed = 27), param_grid = param_step_by_step_test1,
                                          scoring = 'roc_auc',n_jobs = 4, iid = False, cv = 5)
gsearch_step_by_step_test1.fit(X_train, y_train)
print(gsearch_step_by_step_test1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_)

# Output:
#     --- ?????? seconds ---
# So, 

print("--- %s seconds ---" % (time.time() - start_time))

# Save the model to disk
dump(model_step_by_step_test1, 'model_step_by_step_test1.joblib')